## 神经网络

我们可以使用torch.nn包构造神经网络。
一个nn.Module包含神经层和一个方法forward(input)，它返回output。

例如对数字图像进行分类的网络：
![](https://pytorch.org/tutorials/_images/mnist.png)
它是一个简单的前馈网络。它接受输入，一个接一个地通过几个层输入，然后最终给出输出。

神经网络的典型训练过程如下：

- 定义具有一些可学习参数（或权重）的神经网络
- 迭代输入数据集
- 通过网络处理输入
- 计算损失（输出距离正确多远）
- 将梯度传播回网络参数
- 通常使用简单的更新规则更新网络的权重： `weight = weight - learning_rate * gradient`

## 定义网络

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        # 卷积核
        self.conv1=nn.Conv2d(1,6,5)
        self.conv2=nn.Conv2d(6,16,5)
        # 计算
        self.fc1=nn.Linear(16*5*5,120)
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,10)
    
    def forward(self,x):
        # 最大池化层 (2,2)
        x=F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x=F.max_pool2d(F.relu(self.conv2(x)),2)
        x=x.view(-1,self.num_flat_features(x))
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x
    def num_flat_features(self,x):
        size=x.size()[1:]
        num_features=1
        for s in size:
            num_features*=s
        return num_features
net=Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


输出网络的参数

In [7]:
params=list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 5, 5])


In [9]:
# input 32x32
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.1461, -0.0914,  0.0625, -0.0329,  0.0677, -0.1319,  0.0288,  0.0997,
          0.0063,  0.0255]], grad_fn=<AddmmBackward>)


In [10]:
net.zero_grad()
out.backward(torch.randn(1, 10))

### 损失函数
损失函数采用（输出，目标）输入，并计算估计输出距目标的距离的值。

nn包下有几种不同的 损失函数。一个简单的损失是：`nn.MSELoss`它计算输入和目标之间的均方误差。

例如：

In [11]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.8668, grad_fn=<MseLossBackward>)


In [ ]:
'''
input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> view -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss
'''

### 反向传播

In [17]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0016,  0.0026, -0.0026,  0.0139, -0.0041, -0.0113])


### 更显权重
实践中使用的最简单的更新规则是随机梯度下降（SGD）：

`weight = weight - learning_rate * gradient`

我们可以使用简单的python代码实现它：

In [19]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [20]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update